In [1]:
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np
import pandas as pd
import dask.array as da
#from tensorflow.python.client import timeline
import dask.dataframe as dd
import scipy.stats as st
import mirsig.util.utility as ut
import mirsig
import re

from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support, f1_score
%matplotlib inline

In [2]:
# row major
# n = numvars
# nv = number of voters, algorithms
# mats = list of all matricies
def borda (mats, n, nv):
    
    for v in range (nv):
        mats[v] = mats[v].argsort(axis = None)
        
    borda_points = np.zeros((n*n))
    for i in range(n*n):
        # i is the current rank star between 0-15
        # rank pint is i/highest rank possible (n*n-1)
        
        for v in range (nv):
            
            borda_points[(mats[v])[i]] = borda_points[(mats[v])[i]] + (i/(n*n-1))
        
         
    return ((borda_points/nv).reshape(n,n))

# Test

In [3]:
# dream_df, column_names = ut.get_data("data/dream4/insilico_size100_2_multifactorial.tsv", sep ='\t')[0:2]
# dream_df, column_names = ut.get_data("PD_data_GEO.csv")[0:2]

In [4]:
dream_df = pd.read_csv("PD_data_GEO.csv", index_col=0)

In [5]:
dream_df.head()

,GSM506039,GSM506040,GSM506041,GSM506042,GSM506043,GSM506044,GSM506045,GSM506046,GSM506048,GSM506059,...,GSM508711,GSM508712,GSM508713,GSM508714,GSM508715,GSM508716,GSM508718,GSM508728,GSM508731,GSM508732
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
10,7.275555,6.933238,7.122556,7.166563,6.688854,7.376252,7.130211,7.311695,6.970355,6.276751,...,7.286284,7.288502,7.028483,7.240654,6.965365,7.229860,7.192786,6.894235,6.479090,7.394017
100,6.992216,6.888190,6.977542,7.360295,6.917253,7.254953,6.892251,6.818134,6.679706,7.193309,...,6.714657,6.870217,7.048082,6.922886,6.779264,6.766457,6.940693,6.715219,6.907370,6.995507
1000,9.174866,9.361395,9.127344,9.028914,9.382840,9.292321,9.622462,9.337737,9.274463,8.949074,...,8.423656,7.903835,7.966204,8.183184,8.215054,8.098942,7.923988,8.187206,7.787492,8.146724
10000,9.053601,9.625884,8.862464,8.759385,9.834909,8.578497,9.418163,9.364837,9.777157,9.869506,...,9.082945,8.526086,8.614087,8.756672,9.194792,8.852102,8.541018,9.769855,9.400733,8.629743
10002,6.937275,6.795463,7.017565,7.110345,6.640140,6.934404,6.860585,6.850633,6.837039,6.449372,...,6.957908,6.934405,6.779368,7.018905,6.851494,6.972869,7.074771,6.714944,6.722539,7.210103


In [6]:
column_names = list(dream_df.index)
dream_df = dream_df.T

In [7]:
column_names

[10,
 100,
 1000,
 10000,
 10002,
 10005,
 10006,
 10007,
 1001,
 10010,
 100101267,
 100126791,
 100127972,
 100128124,
 100128640,
 100129250,
 100129271,
 100129361,
 100129460,
 100129503,
 100129528,
 10013,
 100130331,
 100130348,
 100130741,
 100131510,
 100131532,
 100131755,
 100131825,
 100131827,
 100132285,
 100132341,
 100133941,
 100134713,
 10014,
 10015,
 10016,
 10017,
 10019,
 1002,
 10020,
 10021,
 10023,
 10024,
 10025,
 10026,
 100272147,
 100272216,
 100272228,
 100288570,
 100289473,
 100289678,
 100290481,
 100293516,
 1003,
 10036,
 10038,
 10039,
 1004,
 10042,
 100422872,
 100423062,
 10043,
 10044,
 10045,
 10046,
 10047,
 10048,
 10049,
 100499177,
 1005,
 10050,
 100505494,
 100505761,
 100505879,
 100505933,
 100506070,
 100506190,
 100506469,
 100506504,
 100506571,
 100506581,
 100506603,
 100506759,
 100506898,
 100506963,
 100507388,
 100507436,
 10051,
 10052,
 100526833,
 10053,
 100533178,
 10054,
 10055,
 10057,
 10058,
 10059,
 1006,
 10060,
 100

In [21]:
# run mrnetb befor clr!!
mrnetb_res= mirsig.mrnetb_cuda(dream_df)

Mrnetb for : 10344
895.001625s sec


In [22]:
mrnetb_res

,0,1,2,3,4,5,6,7,8,9,...,10334,10335,10336,10337,10338,10339,10340,10341,10342,10343
0,0.000000,0.000000,0.000000,0.012069,0.095125,0.000000,0.059990,0.033628,0.000000,0.000000,...,0.000000,0.000000,0.017276,0.024708,0.000000,0.000000,0.144831,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.015419,0.000000,0.000000,0.000000,0.138884,0.060864,...,0.000000,0.009812,0.044696,0.000000,0.000000,0.000000,0.000000,0.039381,0.094019,0.034785
2,0.000000,0.000000,0.000000,0.002033,0.000000,0.000000,0.004282,0.000000,0.000000,0.000000,...,0.025991,0.000000,0.008081,0.000000,0.000000,0.000000,0.000000,0.015576,0.006669,0.000000
3,0.012069,0.000000,0.002033,0.000000,0.000000,0.000000,0.171453,0.077923,0.000000,0.000000,...,0.174823,0.000000,0.000000,0.026496,0.142779,0.011259,0.000000,0.079307,0.000000,0.000000
4,0.095125,0.015419,0.000000,0.000000,0.000000,0.052621,0.011728,0.000000,0.129319,0.000000,...,0.000000,0.130268,0.128893,0.000000,0.000000,0.000000,0.129769,0.000000,0.013291,0.053269
5,0.000000,0.000000,0.000000,0.000000,0.052621,0.000000,0.000000,0.000000,0.112946,0.009554,...,0.000000,0.028029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039795,0.094763
6,0.059990,0.000000,0.004282,0.171453,0.011728,0.000000,0.000000,0.037444,0.000000,0.000000,...,0.070851,0.000000,0.004235,0.026363,0.122388,0.000000,0.015697,0.029808,0.000000,0.000000
7,0.033628,0.000000,0.000000,0.077923,0.000000,0.000000,0.037444,0.000000,0.000000,0.019527,...,0.045548,0.000000,0.000000,0.059973,0.000000,0.060788,0.000893,0.069418,0.031909,0.000000
8,0.000000,0.138884,0.000000,0.000000,0.129319,0.112946,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.118879,0.002861,0.000000,0.000000,0.025975,0.050354,0.046219,0.114365,0.140051
9,0.000000,0.060864,0.000000,0.000000,0.000000,0.009554,0.000000,0.019527,0.000000,0.000000,...,0.000000,0.060483,0.000321,0.000000,0.000000,0.000000,0.000000,0.003149,0.141803,0.028177


In [23]:
clr_res = mirsig.clr(dream_df.values,128,16)

In [24]:
genie_res = mirsig.parallel_genie(dream_df)

100%

In [25]:
genie_no_dir = pd.DataFrame(np.maximum(genie_res.values, genie_res.values.T)) 

In [26]:
spearman_res = mirsig.spearman_df(dream_df)

In [27]:
pearson_res = mirsig.pearson_df(dream_df)

In [28]:
#mrnetb_res= mirsig.mrnetb_cuda()
pearson_res = pearson_res.abs()

In [29]:
spearman_res = spearman_res.abs()

In [30]:
kendall_res = mirsig.kendal_taub_cuda(dream_df)

Kendall for : 10344
16.454732s sec


In [31]:
borda_res = borda([mrnetb_res.values,
                   clr_res,
                   genie_res.values,
                   kendall_res.values,
                   spearman_res.values,
                   pearson_res.values],
                   pearson_res.shape[0], 6)
np.fill_diagonal(borda_res, 0)

In [32]:
np.save('data', borda_res)

In [8]:
borda_res = np.load('data.npy')

In [12]:
dS = pd.DataFrame (borda_res, columns= column_names, index= column_names )
edgeList = dS.stack().reset_index()
edgeList=edgeList[edgeList[0]>0.9]
edgeList[['level_0','level_1'] ].to_csv('PD_network_edgelist.tsv', sep='\t', header=False, index=False)